# Extraccion datos PDF (Despiece motor)  

In [63]:
import pdfplumber
import re
import csv

In [64]:
def extraerCabecera(data):
    cabecera = list()
    div = data.split(" ")
    for i in range(0, len(div)):
        if i == 1:
            column = div[i]
        elif i == 2:
            column = column + ' ' + div[i]
            cabecera.append(column)
        elif div[i] == 'MODEL/TYPE' or div[i] == 'FN' or div[i] == 'MEASUREMENT' or div[i] == 'STANDARD':
            continue
        else:
            cabecera.append(div[i])
    return cabecera
    
def extraerCuerpo(data):
    grupos = []
    txt = re.search(r"^([0-9]+)\s([0-9]+)\s(.{1,}\w?)\s([1-9]+\.[0-9]+|\d{1,4})\s(PC)\s?(.{1,}G|KG| )?", data)
    for i in range(1,7):
        try:
            grupos.append(txt.group(i))
        except AttributeError:
            grupos.append('')
    return grupos  

In [65]:
pdf_file = "data.pdf"
tables={}
otro = []
upc = ['CRANKCASE', 'CVR CYL INSP PORT REM MT 8V HE/REM MT 12','OIL FILLER NECK', 'OIL PAN FLAT (NPTF)']
with pdfplumber.open(pdf_file) as pdf:
    pages = pdf.pages
    for i in range(20):#enumerate(pages):
        txt = pages[i].extract_text()
        sub = re.search(r".{1,15}([a-zA-Z0-9].{1,})\S[A-Z]\/", txt)
        kw = str(sub.group(1)[:-1])
        try:
            data = pages[i].extract_tables()[0]
            if kw in upc:
                if kw in tables:
                    for z in data[1:]:
                        salida = extraerCuerpo(z[0])
                        tables[kw].append(salida)
                else:
                    cabecera = extraerCabecera(data[0][0])
                    tables[kw] = [cabecera]
                    for z in data[1:]:
                        salida = extraerCuerpo(z[0])
                        tables[kw].append(salida)
        except IndexError:
            continue
print(tables)
#for kw in tables:
#    word = str(kw)
#    word = word.replace(" ", "_")
#    word = word.replace("/", "_")
#    with open(word + '.csv', mode="w") as csv_file:
#        writer = csv.writer(csv_file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#
#        for row in tables[kw]:
#            writer.writerow(row)

#   POS     PART NRO    DESIGN          QTY                 UNIT      WEIGHT
#^([0-9]+)\s([0-9]+)\s(.{1,}\w?)\s([1-9]+\.[0-9]+|\d{1,4})\s(PC)\s?(.{1,}G|KG| )?

{'CRANKCASE': [['POS.', 'PART NO.', 'DESIGNATION', 'QTY.', 'UNITS', 'WEIGHT'], ['1', '735084050000', 'PLUG ', '1.000', 'PC', '103,600 G'], ['2', '700429055001', 'O-RING ', '1.000', 'PC', '8,110 G'], ['3', '700429042000', 'O-RING ', '1.000', 'PC', '6,400 G'], ['4', '700429018005', 'O-RING ', '8.000', 'PC', '1,900 G'], ['5', '5269970345', 'SEAL STRIP ', '1.000', 'PC', '145 G'], ['6', '5260101605', 'CRANKCASE ', '1.000', 'PC', '1.611 KG'], ['7', '5260101114', 'COVER ', '1.000', 'PC', None], ['8', '5249950059', 'STRAP ', '1.000', 'PC', '72 G'], ['9', '5240111380', 'GASKET ', '5.000', 'PC', None], ['10', '5240110732', 'COVER CYL BLOCK ACCESS PORT ', '5.000', 'PC', '2,740 KG'], ['11', '5240110440', 'FILLER PIECE STRIP ', '1.000', 'PC', '2,900 KG'], ['12', '5240110159', 'RING CYL LINER CARBON REMOVAL ', '12.000', 'PC', '216 G'], ['13', '007603014102', 'WASHER A COP ', '1.000', 'PC', '1,100 G'], ['14', '007349010002', 'WASHER ', '1.000', 'PC', '11,130 G'], ['15', '0049976736', 'VALVE DRAIN M18